In [43]:
import pandas as pd
import yfinance  as yf
import datetime
import numpy as np
from sklearn.linear_model import LinearRegression

In [44]:
nifty50_symbols = [
    "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "HDFC.NS", "INFY.NS", "ICICIBANK.NS",
    "KOTAKBANK.NS", "HCLTECH.NS", "SBIN.NS", "BHARTIARTL.NS", "ASIANPAINT.NS", "AXISBANK.NS",
    "BAJAJFINSV.NS", "BAJFINANCE.NS", "DIVISLAB.NS", "DRREDDY.NS", "GRASIM.NS", "HINDALCO.NS",
    "HINDUNILVR.NS", "IOC.NS", "ITC.NS", "JSWSTEEL.NS", "LARSEN.NS", "M&M.NS", "MARUTI.NS",
    "NESTLEIND.NS", "NTPC.NS", "ONGC.NS", "POWERGRID.NS", "TATASTEEL.NS", "TITAN.NS",
    "ULTRACEMCO.NS", "UPST.NS", "WIPRO.NS", "SHREECEM.NS", "ADANIPORTS.NS", "BPCL.NS",
    "CIPLA.NS", "COALINDIA.NS", "EICHERMOT.NS", "HEROMOTOCO.NS", "HINDZINC.NS", "GAIL.NS",
    "NAUKRI.NS", "INDUSINDBK.NS", "UPL.NS", "TECHM.NS", "BRITANNIA.NS", "BAJAJ_AUTO.NS"
]

nifty50_data = {}

def stock_data_maker(symbols, interval="1d", period="90d"):
    nifty50_data = {}
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        stock_data = stock.history(interval=interval, period=period)
        nifty50_data[symbol] = stock_data
    return nifty50_data




In [53]:
#now we need to rank the stocks

 # Rank stocks based on volatility-adjusted momentum
def rank_stocks():
    stock_array = np.zeros((1, 2))

    for symbol in nifty50_data:
        stock_data = nifty50_data[symbol]

        # Skip the stock if the data is empty or contains NaN values
        if stock_data.empty or stock_data['Close'].isna().any():
            continue

        prices = np.log(stock_data['Close'])
        dates = stock_data.index.map(lambda x: x.toordinal()).to_numpy().reshape(-1, 1)

        # Remove NaN values from prices and dates
        valid_indices = np.logical_not(np.isnan(prices))
        prices = prices[valid_indices]
        dates = dates[valid_indices]

        # Fit linear regression model
        regression = LinearRegression().fit(dates, prices)
        R2 = regression.score(dates, prices)
        slope = regression.coef_[0]
        annualized_slope = (np.exp(slope) - 1) * 252
        volatility_adjusted_momentum = annualized_slope * R2
        new_row = np.array([symbol, (volatility_adjusted_momentum) ** 2])
        stock_array = np.vstack((stock_array, new_row))

    stock_array = np.delete(stock_array, 0, axis=0)
    stock_array_rank = stock_array[stock_array[:, 1].argsort()[::-1]]
    stock_symbols_rank = np.delete(stock_array_rank, 1, axis=1)
    return stock_symbols_rank

   
print(rank_stocks())

[['GRASIM.NS']
 ['IOC.NS']
 ['KOTAKBANK.NS']
 ['ONGC.NS']
 ['TCS.NS']
 ['ADANIPORTS.NS']
 ['RELIANCE.NS']
 ['BPCL.NS']
 ['HCLTECH.NS']
 ['CIPLA.NS']
 ['UPL.NS']
 ['SHREECEM.NS']
 ['ASIANPAINT.NS']
 ['ITC.NS']
 ['TATASTEEL.NS']
 ['COALINDIA.NS']
 ['BRITANNIA.NS']
 ['M&M.NS']
 ['WIPRO.NS']
 ['NTPC.NS']
 ['DIVISLAB.NS']
 ['INDUSINDBK.NS']
 ['HINDZINC.NS']
 ['EICHERMOT.NS']
 ['AXISBANK.NS']
 ['ICICIBANK.NS']
 ['HINDUNILVR.NS']
 ['NESTLEIND.NS']
 ['INFY.NS']
 ['HDFCBANK.NS']
 ['HDFC.NS']
 ['TECHM.NS']
 ['SBIN.NS']
 ['POWERGRID.NS']
 ['JSWSTEEL.NS']
 ['BHARTIARTL.NS']
 ['ULTRACEMCO.NS']
 ['MARUTI.NS']
 ['DRREDDY.NS']
 ['NAUKRI.NS']
 ['GAIL.NS']
 ['TITAN.NS']
 ['HINDALCO.NS']
 ['HEROMOTOCO.NS']]


In [46]:
#sorting this numpy array to get ranked stocks

# Account value and index conditions
account_value = int(input("Enter the account value: "))
#MDF for the market 
index_buying_condition = 1
nifty50_ticker = '^NSEI'
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=200)
nifty50 = yf.download(nifty50_ticker, start=start_date, end=end_date)
nifty50['200_day_MA'] = nifty50['Close'].rolling(window=200).mean()
MDF_market= yf.download(nifty50_ticker,start=start_date,end=end_date)
today_200_day_MA = nifty50['200_day_MA'].iloc[-1]
today_open = nifty50['Open'].iloc[-2]

if today_open < today_200_day_MA:
    index_buying_condition = 0


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [47]:
# Stock disqualifier
def disqualify_stocks(stock_symbols_rank):
    disqualified_stocks = []
    for symbol in stock_symbols_rank:
        symbol_str = str(symbol[0])  # Convert symbol to string
        stock_data = nifty50_data[symbol_str]
        stock_data['100_day_MA'] = stock_data['Close'].rolling(window=100).mean()

        if stock_data['Open'].iloc[-2] < stock_data['100_day_MA'].iloc[-1] or \
                stock_data['Open'].iloc[-2] > stock_data['Close'].iloc[-2] * 1.15:
            disqualified_stocks.append(symbol_str)

    return disqualified_stocks


In [48]:
def calculate_atr(stock_data, window=20):
    high = stock_data['High']
    low = stock_data['Low']
    close = stock_data['Close']
    tr = pd.DataFrame(index=stock_data.index)

    tr['TR1'] = high - low
    tr['TR2'] = abs(high - close.shift())
    tr['TR3'] = abs(low - close.shift())
    tr['TR'] = tr[['TR1', 'TR2', 'TR3']].max(axis=1)
    
    atr = tr['TR'].rolling(window=window).mean()
    return atr

In [49]:
# Construct the initial portfolio
portfolio = []
available_cash = account_value

ranked_stocks = rank_stocks()
disqualified_stocks = disqualify_stocks(ranked_stocks)

for symbol in ranked_stocks:
    symbol_str = str(symbol[0])  # Convert symbol to string
    if symbol_str not in disqualified_stocks:
        stock_data = nifty50_data[symbol_str]
        price = stock_data['Close'].iloc[-2]
        position_size = (available_cash * 0.001) / calculate_atr(stock_data, 20).iloc[-2]

        if position_size > 0:
            portfolio.append({'symbol': symbol, 'price': price, 'size': position_size})
            available_cash -= price * position_size

        if available_cash <= 0:
            break

In [57]:
print(calculate_atr(nifty50_data['GRASIM.NS'],20))

Date
2022-01-10 00:00:00+05:30          NaN
2022-01-11 00:00:00+05:30          NaN
2022-01-12 00:00:00+05:30          NaN
2022-01-13 00:00:00+05:30          NaN
2022-01-14 00:00:00+05:30          NaN
                               ...    
2023-06-22 00:00:00+05:30    33.044995
2023-06-23 00:00:00+05:30    32.457501
2023-06-26 00:00:00+05:30    32.247504
2023-06-27 00:00:00+05:30    32.430005
2023-06-30 00:00:00+05:30    31.922504
Name: TR, Length: 364, dtype: float64


In [50]:
def rebalance_portfolio():
    for position in portfolio:
        symbol = position['symbol']
        stock_data = nifty50_data[symbol]
        price = stock_data['Close'].iloc[-2]
        position_size = (account_value * 0.001) / calculate_atr(stock_data, 20).iloc[-2]
        target_size = (account_value * 0.001) / calculate_atr(stock_data, 20).iloc[-1]

        if abs(position_size - target_size) >= 0.01:
            position['size'] = target_size
            print(f"Rebalanced {symbol} position from {position_size:.2f} to {target_size:.2f}")


In [51]:
def trade():
    today = datetime.date.today()
    if today.weekday() == 2:  # Only trade on Wednesdays
        if len(portfolio) > 0:
            # Check for sell signals
            for position in portfolio:
                symbol = position['symbol']
                stock_data = nifty50_data[symbol]

                if symbol not in ranked_stocks[:int(len(ranked_stocks) * 0.2)] or \
                        stock_data['Open'].iloc[-2] < stock_data['100_day_MA'].iloc[-1] or \
                        stock_data['Open'].iloc[-2] > stock_data['Close'].iloc[-2] * 1.15 or \
                        symbol not in nifty50_data.keys():
                    print(f"Sold {symbol} position")
                    portfolio.remove(position)
                    available_cash += position['price'] * position['size']

            # Check for buy signals
            if available_cash > 0 and index_buying_condition:
                for symbol in ranked_stocks:
                    if symbol not in disqualified_stocks and symbol not in [p['symbol'] for p in portfolio]:
                        stock_data = nifty50_data[symbol]
                        price = stock_data['Close'].iloc[-2]
                        position_size = (available_cash * 0.001) / calculate_atr(stock_data, 20).iloc[-2]

                        if position_size > 0:
                            portfolio.append({'symbol': symbol, 'price': price, 'size': position_size})
                            available_cash -= price * position_size

                        if available_cash <= 0:
                            break

        # Rebalance positions every second Wednesday
        if today.weekday() % 14 == 0:
            rebalance_portfolio()


In [52]:

import matplotlib.pyplot as plt



end_date = datetime.datetime(2023, 5, 20) 
start_date = end_date - datetime.timedelta(days=365)
portfolio_sizes = []
while start_date < end_date:
    nifty50_data = stock_data_maker(nifty50_symbols, interval="1d", period="365d")
    ranked_stocks = rank_stocks()
    disqualified_stocks = disqualify_stocks(ranked_stocks)
    portfolio_size = trade()
    portfolio_sizes.append(portfolio_size)

    start_date += datetime.timedelta(days=7)

LARSEN.NS: No data found, symbol may be delisted
UPST.NS: No data found, symbol may be delisted
BAJAJ_AUTO.NS: No data found, symbol may be delisted
LARSEN.NS: No data found, symbol may be delisted
UPST.NS: No data found, symbol may be delisted
BAJAJ_AUTO.NS: No data found, symbol may be delisted
LARSEN.NS: No data found, symbol may be delisted
UPST.NS: No data found, symbol may be delisted
BAJAJ_AUTO.NS: No data found, symbol may be delisted
LARSEN.NS: No data found, symbol may be delisted
UPST.NS: No data found, symbol may be delisted
BAJAJ_AUTO.NS: No data found, symbol may be delisted
LARSEN.NS: No data found, symbol may be delisted
UPST.NS: No data found, symbol may be delisted
BAJAJ_AUTO.NS: No data found, symbol may be delisted
LARSEN.NS: No data found, symbol may be delisted
UPST.NS: No data found, symbol may be delisted
BAJAJ_AUTO.NS: No data found, symbol may be delisted
LARSEN.NS: No data found, symbol may be delisted
UPST.NS: No data found, symbol may be delisted
BAJAJ_AUTO

KeyboardInterrupt: 